# Aug 22 - Emulate Single Wavelength

GP emulation of 6S for a fixed wavelength.

In [ ]:
from typing import Final

import alive_progress
import matplotlib.pyplot as plt
import numpy as np
import rtm_wrapper.parameters as rtm_param
import scipy.stats.qmc as sci_qmc
import sklearn.gaussian_process as sklearn_gp
import sklearn.pipeline
import sklearn.preprocessing as sklearn_pre
from rtm_wrapper.engines.sixs import PySixSEngine, pysixs_default_inputs
from rtm_wrapper.execution import ConcurrentExecutor
from rtm_wrapper.simulation import SweepSimulation


def unit2range(arr: np.ndarray, bot: float, top: float) -> np.ndarray:
    return arr * (top - bot) + bot

## Set wavelength and input parameter ranges

In [ ]:
# Fixed wavelength to simulate.
WAVELENGTH: Final = 0.59  # micrometers

# Atmosphere parameter ranges to simulate.
OZONE_RANGE: Final = (0.25, 0.45)  # cm-atm
WATER_RANGE: Final = (1, 4)  # g/cm^2
AOT_RANGE: Final = (0.05, 0.5)  # 1

# Run true 6S simulation

## Sample atmosphere input ranges

In [ ]:
# Number of LHS samples to draw.
NUM_SAMPLES: Final = 20

# Draw LHS samples.
rng = np.random.default_rng(2023_08_21)
lhs_sampler = sci_qmc.LatinHypercube(d=3, seed=rng)
raw_samples = lhs_sampler.random(NUM_SAMPLES)

# Draw Poisson disk samples
# pd_sampler = sci_qmc.PoissonDisk(d=2, seed=rng, radius=0.18)
# raw_samples = pd_sampler.random(NUM_SAMPLES)
# assert len(raw_samples) == NUM_SAMPLES, "failed to draw enough samples - try decreasing radius"

# Rescale LHS samples to parameter ranges.
ozone_samples = unit2range(raw_samples[:, 0], *OZONE_RANGE)
water_samples = unit2range(raw_samples[:, 1], *WATER_RANGE)
# aot_samples = unit2range(raw_samples[:, 1], *AOT_RANGE)

## Plot atmosphere input samples

In [ ]:
# fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8,8))
# ax.scatter(ozone_samples, water_samples, aot_samples)

fig, ax = plt.subplots(figsize=(8, 6))
# fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8,8))
ax.scatter(ozone_samples, water_samples)
# ax.scatter(ozone_samples, water_samples, aot_samples)
ax.set_xlim(OZONE_RANGE)
ax.set_ylim(WATER_RANGE)
# ax.set_zlim(AOT_RANGE)
ax.set_xlabel("Ozone Column ($cm\cdot atm$)")
ax.set_ylabel("Water Column ($g/cm^2$)")
# ax.set_zlabel("AOT")
ax.set_title("Atmosphere Input LHS Samples")
fig.tight_layout()

## Perform simulation

In [ ]:
sweep = SweepSimulation(
    {
        "lhs": {
            "atmosphere.ozone": ozone_samples,
            "atmosphere.water": water_samples,
            # "aerosol_profile.aot": aot_samples,
        },
    },
    base=pysixs_default_inputs().replace(
        atmosphere=rtm_param.AtmosphereWaterOzone(),
        # aerosol_profile=rtm_param.AerosolAOTSingleLayer(profile="Maritime", height=5),
        wavelength__value=WAVELENGTH,
    ),
)

engine = PySixSEngine()
runner = ConcurrentExecutor(max_workers=16)

with alive_progress.alive_bar(sweep.sweep_size, force_tty=True) as bar:
    runner.run(sweep, engine, step_callback=lambda _: bar())

train_results = runner.collect_results()
train_output = train_results.apparent_radiance

display(train_results)

## Plot scatter of outputs at each input

In [ ]:
color_source = train_output.values
color = (color_source - color_source.min()) / (color_source.max() - color_source.min())

# fig, ax =  plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8,8))
# art = ax.scatter(
#     train_results.coords["atmosphere.ozone"].values,
#     train_results.coords["atmosphere.water"].values,
#     train_results.coords["aerosol_profile.aot"].values,
#     c=color,
#     s=90,
#     cmap="viridis",
# )
fig, ax = plt.subplots(figsize=(8, 6))
art = ax.scatter(
    train_results.coords["atmosphere.ozone"].values,
    train_results.coords["atmosphere.water"].values,
    c=color,
    s=90,
    cmap="viridis",
)
n_ticks = 7
cbar = fig.colorbar(art, ticks=np.linspace(0, 1, n_ticks))
cbar.ax.set_yticklabels(
    np.round(np.linspace(color_source.min(), color_source.max(), n_ticks), 2)
)
ax.set_title(f"Apparent Radiance of {WAVELENGTH*1000:.0f}nm")
ax.set_xlabel("Ozone Column ($cm-atm$)")
ax.set_ylabel("Water Column ($g/cm^2$)")
fig.tight_layout()

# Train Emulator

## Extract training arrays

In [ ]:
x_train = np.stack(
    [
        train_results.coords["atmosphere.ozone"].values,
        train_results.coords["atmosphere.water"].values,
        # train_results.coords["aerosol_profile.aot"].values,
    ],
    axis=-1,
)
y_train = train_output.values.reshape(-1, 1)
print(f"{x_train.shape=}, {y_train.shape=}")

## Create GP model

In [ ]:
kernel = 1.0 * sklearn_gp.kernels.RBF() # + sklearn_gp.kernels.WhiteKernel()
gaussian_process = sklearn_gp.GaussianProcessRegressor(
    kernel=kernel,
    n_restarts_optimizer=20,
    alpha=1e-2,
    # alpha=1,
    # Normalize targets to zero means, unit variance.
    normalize_y=True,
)

pipeline = sklearn.pipeline.Pipeline(
    [
        # Rescale input features to [0, 1].
        ("scale", sklearn_pre.MinMaxScaler()),
        ("gp", gaussian_process),
    ]
)
display(pipeline)
display(pipeline.named_steps["gp"].kernel.hyperparameters)

## Fit model

In [ ]:
pipeline.fit(x_train, y_train)
display(pipeline.named_steps["gp"].kernel_)

## Plot marginal likelihood surface

In [ ]:
# Extract fit hyperparameter values.
fit_theta = pipeline.named_steps["gp"].kernel_.theta

# Indices of the two kernel hyperparameters to vary and plot MLL over. 
plot_hyper_idx = [0, 1]
plot_hyper_names = [gaussian_process.kernel.hyperparameters[idx].name for idx in plot_hyper_idx]

# Hyperparameter ranges to compute marginal likelihood over.
# Natural log scaled, and centered about fit hyperparameter values found above.
log_sweep_0 = np.log(10) * np.linspace(-4, 10, 50) + fit_theta[plot_hyper_idx[0]]
log_sweep_1 = np.log(10) * np.linspace(-4, 4, 50) + fit_theta[plot_hyper_idx[1]]


mesh_hyper_0, mesh_hyper_1 = np.meshgrid(log_sweep_0, log_sweep_1)
# Preallocate array for likelihood at each hyperparameter combination.
log_marginal_likelihoods = np.zeros(mesh_hyper_0.shape)

# Compute MLL for each hyperparameter combination.
for hyper_0, hyper_1, out in np.nditer(
    [mesh_hyper_0, mesh_hyper_1, log_marginal_likelihoods],
    op_flags=[["readonly"], ["readonly"], ["writeonly"]],
):
    theta = fit_theta.copy()
    theta[plot_hyper_idx[0]] = hyper_0
    theta[plot_hyper_idx[1]] = hyper_1
    out[...] = gaussian_process.log_marginal_likelihood(theta)

# Plot MLL contours.
fig, ax = plt.subplots()
ax.set_xscale("log")
ax.set_yscale("log")
# Pick contour levels. Increase level density near max to better show peaks.
peak_switch = np.percentile(log_marginal_likelihoods, 70)
levels = np.hstack(
    (
        np.linspace(log_marginal_likelihoods.min(), peak_switch, 30)[:-1],
        np.linspace(peak_switch, log_marginal_likelihoods.max(), 20),
    )
)
# levels = 30
art = ax.contour(
    np.exp(mesh_hyper_0), np.exp(mesh_hyper_1), log_marginal_likelihoods, levels
)
ax.plot(*np.exp(fit_theta), "x")
ax.set_xlabel(plot_hyper_names[0])
ax.set_ylabel(plot_hyper_names[1])
ax.set_title("Marginal Likelihood vs Hyperparameters")
fig.tight_layout()

# Plot 3D MLL surface.
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8, 8))
ax.computed_zorder = False  # Prevent surface from hiding point, https://stackoverflow.com/q/51241367/11082165
ax.view_init(elev=30, azim=-135)
zlims = ax.get_zlim()
ax.scatter(
    [fit_theta[0] / np.log(10)],
    [fit_theta[1] / np.log(10)],
    [gaussian_process.log_marginal_likelihood(fit_theta)],
    c="r",
    s=5,
    zorder=2,
)
ax.plot_surface(
    mesh_hyper_0 / np.log(10),
    mesh_hyper_1 / np.log(10),
    log_marginal_likelihoods,
    # cmap="coolwarm",
    zorder=1,
)
ax.contour(
    mesh_hyper_0 / np.log(10),
    mesh_hyper_1 / np.log(10),
    log_marginal_likelihoods,
    levels=levels,
    zorder=3,
)

ax.set_xlabel(f"log10({plot_hyper_names[0]})")
ax.set_ylabel(f"log10({plot_hyper_names[1]})")
fig.supylabel("log mll")
ax.set_title("Marginal Likelihood vs Hyperparameters")
fig.tight_layout()

# Asses Emulator

## Generate test data

In [ ]:
ozone_test = np.linspace(*OZONE_RANGE, 30)
water_test = np.linspace(*WATER_RANGE, 31)

## Obtain actual sim results for test data

In [ ]:
sweep = SweepSimulation(
    {
        "atmosphere.ozone": ozone_test,
        "atmosphere.water": water_test,
    },
    base=pysixs_default_inputs().replace(
        atmosphere=rtm_param.AtmosphereWaterOzone(),
        wavelength__value=WAVELENGTH,
    ),
)

engine = PySixSEngine()
runner = ConcurrentExecutor(max_workers=16)
with alive_progress.alive_bar(sweep.sweep_size, force_tty=True) as bar:
    runner.run(sweep, engine, step_callback=lambda _: bar())

test_results = runner.collect_results()
test_output = test_results.apparent_radiance

display(test_output)

## Extract test arrays

In [ ]:
mesh_ozone, mesh_water = np.meshgrid(
    test_results.coords["atmosphere.ozone"].values,
    test_results.coords["atmosphere.water"].values,
    indexing="ij",
)

x_test = np.hstack((mesh_ozone.reshape(-1, 1), mesh_water.reshape(-1, 1)))
y_test = test_output.values.reshape(-1, 1)

print(f"{x_test.shape=}, {y_test.shape=}")

## Evaluate model on test data

In [ ]:
pred_mean, pred_std = pipeline.predict(x_test, return_std=True)
pred_error = y_test - pred_mean.reshape(-1, 1)

pred_mean = pred_mean.reshape(mesh_ozone.shape)
pred_std = pred_std.reshape(mesh_ozone.shape)
pred_error = pred_error.reshape(mesh_ozone.shape)
y_test_shaped = y_test.reshape(mesh_ozone.shape)

## Compute metrics

In [ ]:
rmse = np.sqrt(np.mean(pred_error**2))

abs_error = np.abs(pred_error)

print(f"RMSE: {rmse:0.2f}")
print(f"Avg abs err: {np.mean(abs_error):0.2f}")
print(f"Max abs err: {np.max(abs_error):0.2f}")
print(f"Avg rel err: {np.mean(abs_error/y_test_shaped):0.2%}")
print(f"Max rel err: {np.max(abs_error/y_test_shaped):0.2%}")

## Plot posterior mean, std, error

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8, 8), sharex="all", sharey="all")

mesh_ozone, mesh_water = np.meshgrid(ozone_test, water_test, indexing="ij")

# Plot predicted mean surface.
ax = axs[0, 0]
art = ax.pcolormesh(mesh_ozone, mesh_water, pred_mean)
ax.plot(
    train_results.coords["atmosphere.ozone"].values,
    train_results.coords["atmosphere.water"].values,
    "o",
    color="k",
    markerfacecolor="none",
)
ax.set_title("Posterior Mean")
cbar = fig.colorbar(art)

# Plot true output surface.
ax = axs[0, 1]
art = ax.pcolormesh(mesh_ozone, mesh_water, y_test_shaped)
ax.plot(
    train_results.coords["atmosphere.ozone"].values,
    train_results.coords["atmosphere.water"].values,
    "o",
    color="k",
    markerfacecolor="none",
)
ax.set_title("True Output")
fig.colorbar(art)

# Plot predicted variance surface.
ax = axs[1, 0]
art = ax.pcolormesh(mesh_ozone, mesh_water, pred_std)
ax.plot(
    train_results.coords["atmosphere.ozone"].values,
    train_results.coords["atmosphere.water"].values,
    "o",
    color="k",
    markerfacecolor="none",
)
ax.set_title("Posterior Std")
fig.colorbar(art)

# Plot error surface.
ax = axs[1, 1]
art = ax.pcolormesh(mesh_ozone, mesh_water, np.abs(pred_error))
ax.plot(
    train_results.coords["atmosphere.ozone"].values,
    train_results.coords["atmosphere.water"].values,
    "o",
    color="k",
    markerfacecolor="none",
)
ax.set_title("Abs. Error")
fig.colorbar(art)

fig.supxlabel("Ozone Column ($cm\cdot atm$)")
fig.supylabel("Water Column ($g/cm^2$)")

fig.tight_layout()